In [3]:
from __future__ import print_function
import sys
from pylsl import StreamInfo, StreamOutlet
import numpy as np

sys.path.append('..')  # help python find cyton.py relative to scripts folder
from openbci import wifi as bci
import logging


SCALE_FACTOR_EEG = (4500000)/24/(2**23-1) #uV/count

logging.basicConfig(filename="test.log", format='%(asctime)s - %(levelname)s : %(message)s', level=logging.DEBUG)
logging.info('---------LOG START-------------')
# If you don't know your IP Address, you can use shield name option
# If you know IP, such as with wifi direct 192.168.4.1, then use ip_address='192.168.4.1'
shield_name = 'OpenBCI-F2C0'
shield_sr = 1000 # Sampling Rate
#shield = bci.OpenBCIWiFi( shield_name=shield_name, log=True, high_speed=True, sample_rate=shield_sr)
shield = bci.OpenBCIWiFi(ip_address='192.168.0.102',shield_name=shield_name, log=True, high_speed=True, sample_rate=shield_sr)
print("WiFi Shield Instantiated")

print("Sample rate ",shield.getSampleRate())
shield.set_sample_rate(shield_sr)
print("Sample rate ",shield.getSampleRate())
shield.test_signal(1) # 1 - enable syntetic square wave, 0 - disable syntethic square wave


# Print to lsl
print("Creating LSL stream for EEG. \nName: OpenBCIEEG1\nID: OpenBCItestEEG1\n")
info_eeg1 = StreamInfo('OpenBCIEEG1', 'EEG', 16, shield_sr, 'float32', 'OpenBCItestEEG1')
print("Creating LSL stream for ACCEL. \nName: OpenBCIACCEL1\nID: OpenBCItestACCEL1\n")
info_accel1 = StreamInfo('OpenBCIACCEL1', 'ACCEL', 3, shield_sr, 'float32', 'OpenBCItestACCEL1')

outlet_eeg1 = StreamOutlet(info_eeg1)
outlet_accel1 = StreamOutlet(info_accel1)

def printData(sample):
    outlet_eeg1.push_sample(np.array(sample.channel_data)*SCALE_FACTOR_EEG)
    outlet_accel1.push_sample(np.array(sample.accel_data))
    #outlet_accel1.push_sample(np.array(sample.sample_number))

shield.start_streaming(printData)
shield.loop()




Welcome to OpenBCI Native WiFi Shield Driver - Please contribute code!
Opened socket on 192.168.0.100:55928
Init WiFi connection with IP: 192.168.0.102
Connected to daisy with 16 channels
WiFi Shield to Python TCP Socket Established
WiFi Shield Instantiated
Sample rate  1000
Success: Sample rate is 1000Hz

Sample rate  1000
Something went wrong while setting signal: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Creating LSL stream for EEG. 
Name: OpenBCIEEG1
ID: OpenBCItestEEG1

Creating LSL stream for ACCEL. 
Name: OpenBCIACCEL1
ID: OpenBCItestACCEL1

Incoming connection from ('192.168.0.102', 19517)


KeyboardInterrupt: 